#### Importing required packages

In [2]:
# Import necessary libraries
import os, wget, tarfile
from g4f.client import Client
# Import necessary libraries
import pandas as pd
from IPython.display import clear_output

#### Fetching the dataset

In [3]:
base_dir = './aclImdb'
sub_dirs =[
    'test/pos',
    'test/neg'
]

# Check if the dataset directory exists
if os.path.exists(base_dir):
    print(f"Dataset already exists.")
else:
    try:
# Download the dataset using wget
        wget.download('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
        print(f"Dataset fetched.")
# Extract the downloaded tar.gz file
        with tarfile.open('aclImdb_v1.tar.gz') as t:
            t.extractall()
        print(f"Dataset extracted.")
    except Exception as e:
        print(f"Dataset download failed.")

Dataset already exists.


In [4]:
# Define a prompt structure for the LLM
def prompt(review):
    return [
        {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of given text.'},
        {'role': 'user', 'content': f'"{review}", is this review positive, or negaitve?, Answer using one word only, either "Poisitve", or "Negative".'}
    ]

In [5]:
# Format the model response to extract sentiment
def format(response):
    r = response.choices[0].message.content.lower()
    if r.find('pos') != -1: return 'Positive'
    elif r.find('neg') != -1: return 'Negative'
    else: return 'Uncertain'

In [6]:
# Instantiate the g4f client to interact with the model
client = Client()

#### Llama-3.1-8B

In [7]:
# Define a function to call the LLaMA 3.1 8B model with a review
def llama318b(review):
# Use the LLM to generate a sentiment response
    response = client.chat.completions.create(
        model='llama-3.1-8b',
        messages=prompt(review),
        max_tokens=8,
        temperature=0.5
    )
    return format(response)

In [8]:
llama318b('It was good!')

'Positive'

#### GPT-4o-Mini

In [9]:
def gpt4oMini(review):
# Use the LLM to generate a sentiment response
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=prompt(review),
        max_tokens=8,
        temperature=0.5
    )
    return format(response)

In [10]:
gpt4oMini('It was good!')

'Positive'

#### Loading the dataset

In [11]:
df_path = 'dataframe.csv'
ids = [0, 0]
# Check if the dataset directory exists
if os.path.exists(df_path):
    df = pd.read_csv(df_path)
    ids[0] = df[df['Label'] == 'Positive']['ID'].max() + 1
    ids[1] = df[df['Label'] == 'Negative']['ID'].max() + 1
    print("Dataframe loaded successfully.")
else:
    df = pd.DataFrame(columns=['ID', 'Rating', 'Review', 'Label', 'Llama-3.1-8B', 'GPT-4o-Mini'])
    print("Dataframe created successfully.")

Dataframe loaded successfully.


In [12]:
def get_reviews():
    reviews = []
    for i, sub_dir in enumerate(sub_dirs):
        dir = f'{base_dir}/{sub_dir}'
        for file in os.listdir(dir):
            if file.startswith(f'{ids[i]}_'):
                with open(f'{dir}/{file}', 'r') as f:
                    reviews.append([int(ids[i]), int(file.split('_')[1][:-4]), 'Positive' if i == 0 else 'Negative', f.read()])
                break
    return reviews

In [20]:
n = 2
for _ in range(int(n/2)):
    for i, review in enumerate(get_reviews()):
        row = pd.DataFrame({
            'ID': review[0],
            'Rating': review[1],
            'Review': review[3],
            'Label': review[2],
            'Llama-3.1-8B': llama318b(review),
            'GPT-4o-Mini': gpt4oMini(review)
        }, index=[0])
        df = pd.concat([df, row], ignore_index=True)
        ids[i] += 1
        
        clear_output()   
        print(f'The dataframe now has {df.shape[0]} reviews.')

df.tail(n)

The dataframe now has 1006 reviews.


,ID,Rating,Review,Label,Llama-3.1-8B,GPT-4o-Mini
1004,502,7,"Interesting twist on the Vampire yarn - fast, ...",Positive,Positive,Positive
1005,502,4,"The film, Heaven's Gate, was a good view, alth...",Negative,Positive,Negative


In [21]:
df = df.filter(['ID', 'Rating', 'Review', 'Label', 'Llama-3.1-8B', 'GPT-4o-Mini'])
df.to_csv('dataframe.csv')

#### Evaluation

In [22]:
TP = [0, 0] # True positives
FP = [0, 0] # False positives
TN = [0, 0] # True negatives
FN = [0, 0] # False negatives
models = ['Llama-3.1-8B', 'GPT-4o-Mini']
results = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1-score'])

for i, row in df.iterrows():
    if row['Label'] == 'Positive':
        if row['Label'] == row['Llama-3.1-8B']:
            TP[0] += 1
        else:
            FP[0] += 1
        if row['Label'] == row['GPT-4o-Mini']:
            TP[1] += 1
        else:
            FP[1] += 1
    else:
       if row['Label'] == row['Llama-3.1-8B']:
            TN[0] += 1
       else:
            FN[0] += 1
       if row['Label'] == row['GPT-4o-Mini']:
            TN[1] += 1
       else:
            FN[1] += 1

for i in range(2):
    accuracy = (TP[i] + TN[i]) / (TP[i] + FP[i] + TN[i] + FN[i])
    precision = TP[i] / (TP[i] + FP[i])
    recall = TP[i] / (TP[i] + FN[i])
    f1_score = (2 * precision * recall) / (precision + recall)

    row = pd.DataFrame({'Accuracy': f'{(accuracy):.4f}',
                        'Precision': f'{(precision):.4f}',
                        'Recall': f'{(recall):.4f}',
                        'F1-score': f'{(f1_score):.4f}'
                       }, index=[models[i]])
    results = pd.concat([results, row])

results

,Accuracy,Precision,Recall,F1-score
Llama-3.1-8B,0.9314,0.9801,0.8931,0.9346
GPT-4o-Mini,0.9364,0.9722,0.9072,0.9386
